<a href="https://colab.research.google.com/github/kuldeepthakare7/Chirp-Birdsong-recognition/blob/main/Final_Model_Chirp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'birdsong-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F19596%2F1292430%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240204T192531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db04b498d8a1a3c9686205871df6ab33290c2f25545dc6262e92c74beee6f2e3b46acf19d502c27c3abb3838223f68adc5ac5cc1c0fd1be92a50b78b47b5477d3f9b3e15b481f65fef132ef7d42932e4cb1bbd840d24302bab20a5cf4ce48cd336caafa839d80cea5e73b41aab1906cf7a6a3fc4953806a1fb79f6717e7e2e51f446f632f4bb726183d5dff18f29676f9b65e03df82b2e2a286659e4a5bdb024a8670cad199257ab18e0ae760c39c80b83cc7a1fca1d90e0982fbaa05c701b05b2b4f763963e24b97e8b4917ba7f611feb03356e7eac19d173d8235ed9fca50f200e341eb45f761e8d4cef445b1194439c0d62732edd0b0343c55d388f3741945,birdcall-check:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F726424%2F1262046%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240204T192531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6bf54a7ccd48fea1f3bb0fedb8cc487b83c2871ec417edb89fd35a13526f0a054cf467808927ae415465b448c35c880ed9d47876a46554abe3480dfd951eeaef20f08910f3c8f67c03fc02745c6667446c1d5305d1592fe92a2c654923d346c2ce4374c3afad3c633e98acdcf83b4cd6b146325b2d622c1d1a0c734e572a016fbec13c65ba954f6997c6b4beb122ee97c84d1571583dc871efa3ab835709471e12511cc6b7c7aa1fc901f26b1029d4cee6b0cd5e95b253bd71b7af8b4883cbdcc78c353bdebfef2a27e0b98bf683e6e1056b0238f4f2d4e0f25aab4365f1bd73c04ef7406b747c7c554683c264107578711221a573e1327986fd59413f789f59,birdcall-resnet50-init-weights:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F725893%2F1264575%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240204T192531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72f69332743c324027c16c8c7f0458d94a9dd7e4388736a94bf38555830b6b6cec76cb04abcb61e1c1d97b844c9cc0e2f9617d76194d345d6545c6b5d91699445405329940da14373ab592c1a75a130499a9a52e7afe026f1671cdac2328727187f747202e5db4795b71d52d58b77dc941deb27034d3bc25e15e4900f650d5ba13d0dde1271a028781d1b6f0edc61a519689350f66c3c47c1da595cefbb84c09b0b759a6819cc71698c96f240329e3ec8c982556548ee2211dcb95e57d72db5b75c29d1bed3e898fa03d870efcb8fd69070777de33957235aa0fce7192c4e76f8c589194dc7cd2a39c5a07274b3b170395dbca47d5d9bcd38532e22b6151552f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv("/kaggle/input/birdsong-recognition/train.csv")

In [ ]:
df.head()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
new_df  = df.isnull().sum()
new_df.sort_values(ascending=False)

In [ ]:
df['rating'].unique()

In [ ]:
df['rating'].isnull().sum()

In [ ]:
df['playback_used'].unique()

In [ ]:
df['playback_used'].isnull().sum()

In [ ]:
df['playback_used'].value_counts()

In [ ]:
df['playback_used'].fillna('no',inplace=True)

In [ ]:
df['playback_used'].isnull().sum()

In [ ]:
le  = LabelEncoder()
df['playback_used'] = le.fit_transform(df['playback_used'])
df['playback_used']


In [ ]:
df['playback_used'].value_counts()

In [ ]:
df['ebird_code'].unique()

In [ ]:
df['ebird_code'].isnull().sum()

In [ ]:
df['ebird_code'].nunique()

In [ ]:
df['channels'].unique()

In [ ]:
df['channels'].isnull().sum()

In [ ]:
df['channels'].value_counts()

In [ ]:
df['channels'] = df['channels'].astype(str).str[0].astype(int)
df['channels']

In [ ]:
df['channels'].unique()

In [ ]:
df['date'].isnull().sum()

In [ ]:
df['date'].unique()

In [ ]:
df['year'] = df['date'].apply(lambda x:x.split('-')[0]).astype(int)
df['month'] = df['date'].apply(lambda x:x.split('-')[1]).astype(int)
df['day'] = df['date'].apply(lambda x:x.split('-')[2]).astype(int)


In [ ]:
df.drop('date', axis=1, inplace=True)


In [ ]:
df.head()

In [ ]:
df['pitch'].isnull().sum()

In [ ]:
df['pitch'].unique()

In [ ]:
df['pitch'].value_counts()

In [ ]:
df['duration'].isnull().sum()

In [ ]:
df['duration'].unique()

In [ ]:
df.head()

In [ ]:
df['filename'].isnull().sum()

In [ ]:
df['filename'].unique()

In [ ]:
df['speed'].isnull().sum()

In [ ]:
df['speed'].unique()

In [ ]:
df['speed'].value_counts()

In [ ]:
df.head()

In [ ]:
df['species'].isnull().sum()

In [ ]:
df['species'].unique()

In [ ]:
df['species'].nunique()

In [ ]:
df['sci_name'].nunique()

In [ ]:
df['number_of_notes'].isnull().sum()

In [ ]:
df['number_of_notes'].unique()

In [ ]:
df.columns

In [ ]:
df['title'].isnull().sum()

In [ ]:
df['title'].unique()

In [ ]:
df['title'].nunique()

In [ ]:
df.drop('title',axis=1,inplace=True)

In [ ]:
df['secondary_labels'].unique()

In [ ]:
df['secondary_labels'].nunique()

In [ ]:
df['secondary_labels'].isnull().sum()

In [ ]:
df.drop('secondary_labels',axis=1,inplace=True)

In [ ]:
df['bird_seen'].isnull().sum()

In [ ]:
df['bird_seen'].unique()

In [ ]:
df['bird_seen'].value_counts()

In [ ]:
df['bird_seen'].fillna('yes',inplace=True)

In [ ]:
df['location'].isnull().sum()

In [ ]:
df['location'].unique()

In [ ]:
df['location'].nunique()

In [ ]:
df['latitude'].isnull().sum()

In [ ]:
df['location'].nunique()

In [ ]:
df = df[df['latitude']!='Not specified']
df = df[df['longitude']!='Not specified']


In [ ]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df['sampling_rate'].isnull().sum()

In [ ]:
df['sampling_rate'].unique()

In [ ]:
df['sampling_rate'].nunique()

In [ ]:
df['sampling_rate'].value_counts()

In [ ]:
df['sampling_rate'] = df['sampling_rate'].apply(lambda x:x.split(' ')[0]).astype(int)

In [ ]:
df['sampling_rate'].unique()

In [ ]:
df['type'].isnull().sum()

In [ ]:
df['type'].unique()

In [ ]:
df['type'].nunique()

In [ ]:
adjusted_type = df['type'].apply(lambda x:x.split(',')).reset_index().explode("type")
adjusted_type = adjusted_type['type'].apply(lambda x:x.strip().lower()).reset_index()
adjusted_type['type'] = adjusted_type['type'].replace('calls','call')
adjusted_type['type'].unique()

In [ ]:
df['elevation'].unique()

In [ ]:
df['elevation'].isnull().sum()

In [ ]:
df['elevation'] = df['elevation'].replace(['? m','Unknown m',' m'],'99999')

In [ ]:
df['elevation'] = df['elevation'].str.replace(',','')

In [ ]:
df['elevation'] = df['elevation'].str.replace('.','')

In [ ]:
df['elevation'] = df['elevation'].str.replace('m','')

In [ ]:
df['bitrate_of_mp3'].unique()

In [ ]:
df['bitrate_of_mp3'].isnull().sum()

In [ ]:
df['bitrate_of_mp3'].value_counts()

In [ ]:
df[df['bitrate_of_mp3']=='3']

In [ ]:
df.drop(df[df['bitrate_of_mp3']=='3'].index,inplace=True)

In [ ]:
df[df['bitrate_of_mp3'].isna()]

In [ ]:
df['bitrate_of_mp3'].fillna('128000 (bps)')

In [ ]:
df['bitrate_of_mp3'].isnull().sum()

In [ ]:
df.columns

In [ ]:
df['file_type'].isnull().sum()

In [ ]:
df['file_type'].unique()

In [ ]:
df['volume'].isnull().sum()

In [ ]:
df['volume'].unique()

In [ ]:
df['background'].isnull().sum()

In [ ]:
df.drop('background',axis=1,inplace=True)

In [ ]:
df['xc_id'].isnull().sum()

In [ ]:
df['xc_id'].unique()

In [ ]:
df['xc_id'].nunique()

In [ ]:
df.drop('xc_id',axis=1,inplace=True)

In [ ]:
df['country'].unique()

In [ ]:
df['country'].isnull().sum()

In [ ]:
df['author'].unique()

In [ ]:
df['author'].nunique()

In [ ]:
df['recordist'].nunique()

In [ ]:
df[df['author']!=df['recordist']]

In [ ]:
df.drop('recordist',axis=1,inplace=True)

In [ ]:
df['length'].isnull().sum()

In [ ]:
df['length'].unique()

In [ ]:
df['min_value'] = df['length'].str.extract(r'(\d+)')
df['min_value'] = pd.to_numeric(df['min_value'],errors = 'coerce')
df['max_value'] = df['length'].str.extract(r'-(\d+)')
df['max_value'] = pd.to_numeric(df['max_value'],errors = 'coerce')
df.head()

In [ ]:
df['min_value'].unique()

In [ ]:
df['max_value'].unique()

In [ ]:
df['time'].isnull().sum()

In [ ]:
df['time'].unique()

In [ ]:
df['time'].nunique()

In [ ]:
df['license'].isnull().sum()

In [ ]:
df['license'].unique()


In [ ]:
le = LabelEncoder()
df['license'] = le.fit_transform(df['license'])
df['license']

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
sns.countplot(x = 'rating',hue='playback_used',data=df,palette='hls')
plt.title('Rating vs Playback used')
plt.xlabel('Rating AND Playback used')
plt.ylabel('count')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
sns.countplot(x = 'rating',hue='pitch',data=df,palette='Set2')
plt.title('Rating vs Pitch')
plt.xlabel('Rating AND Pitch')
plt.ylabel('count')
plt.show()

In [ ]:
numbrs = df['pitch'].value_counts()
plt.figure(figsize=(8,8))
plt.pie(numbrs,labels=None,autopct='%1.1f%%')
plt.title('Pitch Distribution')
plt.legend(numbrs.index,title='Categories',loc='upper right',bbox_to_anchor=(1,0,0.5,1))
plt.show()

In [ ]:
import librosa
import librosa.display
import soundfile as sf
import IPython.display as ipd

In [ ]:
a1 = "/kaggle/input/birdsong-recognition/train_audio/ameavo/XC178760.mp3"
a2 = "/kaggle/input/birdsong-recognition/train_audio/ameavo/XC178761.mp3"
b1 = "/kaggle/input/birdsong-recognition/train_audio/calgul/XC161035.mp3"
b2 = "/kaggle/input/birdsong-recognition/train_audio/calgul/XC161036.mp3"
c1 = "/kaggle/input/birdsong-recognition/train_audio/fiscro/XC130581.mp3"
c2 = "/kaggle/input/birdsong-recognition/train_audio/fiscro/XC130579.mp3"
d1 = "/kaggle/input/birdsong-recognition/train_audio/magwar/XC111149.mp3"
d2 = "/kaggle/input/birdsong-recognition/train_audio/magwar/XC134515.mp3"
e1 = "/kaggle/input/birdsong-recognition/train_audio/swahaw/XC325718.mp3"
e2 = "/kaggle/input/birdsong-recognition/train_audio/swahaw/XC267710.mp3"

In [ ]:
y1,sr1 = librosa.load(a1)
y2,sr2 = librosa.load(a2)
y3,sr3 = librosa.load(b1)
y4,sr4 = librosa.load(b2)
y5,sr5 = librosa.load(c1)
y6,sr6 = librosa.load(c2)
y7,sr7 = librosa.load(d1)
y8,sr8 = librosa.load(d2)
y9,sr9 = librosa.load(e1)
y10,sr10 = librosa.load(e2)


In [ ]:
ipd.Audio(y1,rate = sr1)

In [ ]:
ipd.Audio(y2,rate = sr2)

In [ ]:
ipd.Audio(y3,rate = sr3)

In [ ]:
ipd.Audio(y4,rate = sr4)

In [ ]:
ipd.Audio(y5,rate = sr5)

In [ ]:
ipd.Audio(y6,rate = sr6)

In [ ]:
ipd.Audio(y7,rate = sr7)

In [ ]:
ipd.Audio(y8,rate = sr8)

In [ ]:
ipd.Audio(y9,rate = sr9)

In [ ]:
ipd.Audio(y10,rate = sr10)

In [ ]:
plt.plot(y1)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y2)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y3)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y4)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y5)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y6)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y7)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y8)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y9)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
plt.plot(y10)
plt.ylabel("Amplitude")
plt.xlabel("Time")

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y1, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr1, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y2, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr2, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y3, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr3, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y4, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr4, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y5, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr5, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y6, hop_length=256)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr6, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y7, hop_length=512)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr7, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y8, hop_length=512)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr8, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y9, hop_length=512)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr9, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y10, hop_length=512)), ref=np.max)
plt.figure(figsize=(8, 4))
librosa.display.specshow(D, sr=sr10, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Audio Signal')
plt.show()

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y1, sr=sr1, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y2, sr=sr2, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y3, sr=sr3, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y4, sr=sr4, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y5, sr=sr5, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y6, sr=sr6, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y7, sr=sr7, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y8, sr=sr8, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y9, sr=sr9, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mel_signal = librosa.feature.melspectrogram(y=y10, sr=sr10, n_fft=1024, hop_length=64)
mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
plt.figure(figsize=(7, 6))
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

In [ ]:
mfcc = librosa.feature.mfcc(y=y1,sr=sr1)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr1, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y2,sr=sr2)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr2, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y3,sr=sr3)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr3, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y4,sr=sr4)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr4, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y5,sr=sr5)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr5, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y6,sr=sr6)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr6, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y7,sr=sr7)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr7, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y8,sr=sr8)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr8, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y9,sr=sr9)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr9, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(y=y10,sr=sr10)
plt.figure(figsize=(8, 4))
librosa.display.specshow(mfcc, sr=sr10, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC of the Audio Signal')
plt.show()

In [ ]:
# By doing the feature extraction we see that mel-spectrogram and mfcc are both very good
# for using in the model. I prefer to use mel-spectrogram.

In [ ]:
import cv2
import audioread
import logging
import os
import random
import time
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from contextlib import contextmanager
from pathlib import Path
from typing import Optional
from fastprogress import progress_bar
from sklearn.metrics import f1_score
from torchvision import models


In [ ]:
def set_seed( seed: int=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backendscudnn.benchmark

def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.seLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger

@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)


In [ ]:
test = pd.read_csv("/kaggle/input/birdcall-check/test.csv")
test_audio = "/kaggle/input/birdcall-check/test_audio"
test.head()

In [ ]:
class ResNet(nn.Module):
    def __init__(self, base_model_name: str, pretrained=False,num_classes=264):
        super().__init__()
        base_model = models.__getattribute__(base_model_name)(pretrained=pretrained)
        layers.append(nn.AdaptiveMaxPool2d(1))
        seld.encoder = nn.sequential(*layers)

        in_features = base_model.fc.in_features
        self.classifier = nn.Sequential(
             nn.Linear(in_fetaures,1024, nn.ReLU(),nn.Dropout(p=0.2)),
             nn.Linear(1024,1024), nn.ReLU(), nn.Dropout(p=02.),
             nn.Linear(1024, num_classes))
    def forward(self, x):
        batch_size = x.size(0)
        x = self.encoder(x).view(batch_size, -1)
        x = self.classifier(x)
        multiclass_proba = F.softmax(x, dim=1)
        multilabel_proba = F.sigmoid(x)
        return {
            "logits": x,
            "multiclass_proba": multiclass_proba,
            "multilabel_proba": multilabel_proba,
        }

In [ ]:
model_config = {
    "base_model_name": "resnet50",
    "pretrained": False,
    "num_classes": 264,
}
melspectrogram_parameters = {
    "n_mels": 128,
    "fmin": 20,
    "fmax":16000,
}
weights_path = "/kaggle/input/birdcall-resnet50-init-weights/best.pth"

In [ ]:
df = pd.read_csv("/kaggle/input/birdsong-recognition/train.csv")
unique = df['ebird_code'].unique()
le = LabelEncoder()
encoded_labels = le.fit_transform(unique)
BIRD_CODE = dict(zip(unique, encoded_labels))
INV_BIRD_CODE = {v :k for k , v in BIRD_CODE.items()}



In [ ]:
def mono_to_color(X: np.ndarray,mean=None,std=None,norm_max=None,norm_min=None,eps=1e-6):
    X = np.stack([X,X,X],axis=-1)

    mean = mean or X.mean()
    X = X-mean
    std = std or X.std()
    Xstd = X/(std+eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or max
    norm_min = norm_min or min

    if((_max-_min) > eps):
        V = Xstd
        V[V< norm_min] = norm_min
        V[V> norm_max] = norm_max
        V = 255*(V-norm_min)/(norm_max-norm_min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(Xstd, dtype=np.uitn8)
    return V

class TestDataset(data.Daateset):
    def __init__(self, df:pd.DataFrame, clip: np.ndarray,img_size=224,melspectrogram_parameters = {}):
        self.df = df
        self.clip = clip
        self.img_size = img_size
        self.melspectrogram_parameters = melspectrogram_parameters
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx:int):
        SR = 32000
        sample = self.df.loc[idx,:]
        site = sample.site
        row_id = sample.row_id

        if site=="site_3":
            y = self.clip.astype(np.float32)
            len_y = len(y)
